In [1]:
import pandas as pd
import numpy as np

# Load the Stata dataset
df = pd.read_stata(r"C:\Users\ngodin\Dropbox\RESEARCH\active_projects\florida_condo\final_datasets\master_datasets\master_dataset_unit_crossection.dta")

# Variables to summarize
vars_of_interest = [
    'total_monthly_hoa_attom_unit', 'num_units_final_assoc', 'num_bldgs_final_assoc',
    'num_stories_final_assoc', 'sqft_attom_unit', 'yr_blt_attom_bldg',
    'miles_to_coast_assoc', 'frac_npexcorp_state_attom_assoc', 'frac_corp_own_attom_assoc',
    'corp_mgmt_city_attom_assoc', 'med_income_nonprimary_assoc',
    'percap_income_prop_zip', 'firststreet_risk_cat_assoc'
]

# Label mapping for LaTeX
var_rename = {
    'total_monthly_hoa_attom_unit': 'Monthly HOA Fee',
    'num_units_final_assoc': 'Units',
    'num_bldgs_final_assoc': 'Buildings',
    'num_stories_final_assoc': 'Stories',
    'sqft_attom_unit': 'Unit Sqft',
    'yr_blt_attom_bldg': 'Year Built',
    'miles_to_coast_assoc': 'Miles to Coast',
    'frac_npexcorp_state_attom_assoc': 'Frac. Out-of-State Owners',
    'frac_corp_own_attom_assoc': 'Frac. Corp Owners',
    'corp_mgmt_city_attom_assoc': 'Corp Mgmt (City)',
    'med_income_nonprimary_assoc': 'Med Income (Nonprimary)',
    'percap_income_prop_zip': 'Per Capita Income (ZIP)',
    'firststreet_risk_cat_assoc': 'Flood Risk (First Street)'
}

# Assign groups
def story_group(n):
    if pd.isna(n): return np.nan
    elif n <= 2: return '1-2'
    elif n <= 4: return '3-4'
    elif n <= 9: return '5-9'
    else: return '10+'

def county_group(c):
    if c in ['MIAMI-DADE', 'BROWARD']:
        return 'Miami/Broward'
    elif pd.isna(c):
        return np.nan
    else:
        return 'Other'

df['StoryGroup'] = df['num_stories_final_assoc'].apply(story_group)
df['CountyGroup'] = df['mm_fips_county_name_attom'].apply(county_group)

# Add 'All' category for story and county
df['StoryGroupAll'] = df['StoryGroup'].fillna('All')
df['CountyGroupAll'] = df['CountyGroup'].fillna('All')

# Define story/county categories
story_groups = ['1-2', '3-4', '5-9', '10+', 'All']
county_groups = ['Miami/Broward', 'Other', 'All']
group_combos = [(s, c) for s in story_groups for c in county_groups]

# Collect summary stats
rows = []

for var in vars_of_interest:
    for stat in ['Mean', 'Median', 'Std. Dev.']:
        row = {'Variable': var_rename.get(var, var), 'Statistic': stat}
        for s, c in group_combos:
            sub = df.copy()
            if s != 'All':
                sub = sub[sub['StoryGroup'] == s]
            if c != 'All':
                sub = sub[sub['CountyGroup'] == c]
            x = sub[var].dropna()
            val = np.nan
            if len(x) > 0:
                if stat == 'Mean':
                    val = x.mean()
                elif stat == 'Median':
                    val = x.median()
                elif stat == 'Std. Dev.':
                    val = x.std()
            col_label = f"{s} | {c}"
            row[col_label] = round(val, 2) if pd.notna(val) else ''
        rows.append(row)

summary_df = pd.DataFrame(rows)

# Sort for cleaner presentation
summary_df = summary_df.sort_values(by=['Variable', 'Statistic'])

summary_df

,Variable,Statistic,1-2 | Miami/Broward,1-2 | Other,1-2 | All,3-4 | Miami/Broward,3-4 | Other,3-4 | All,5-9 | Miami/Broward,5-9 | Other,5-9 | All,10+ | Miami/Broward,10+ | Other,10+ | All,All | Miami/Broward,All | Other,All | All
6,Buildings,Mean,3.660000,4.940000,4.630000,5.880000,8.210000,7.250000,8.560000,6.430000,7.480000,2.720000,3.020000,2.810000,4.370000,5.410000,4.930000
7,Buildings,Median,1.000000,1.000000,1.000000,2.000000,2.000000,2.000000,2.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
8,Buildings,Std. Dev.,6.590000,12.440000,11.340000,8.780000,16.150000,13.660000,16.050000,14.490000,15.310000,3.250000,4.730000,3.750000,8.830000,12.630000,11.040000
27,Corp Mgmt (City),Mean,0.540000,0.600000,0.580000,0.650000,0.630000,0.640000,0.620000,0.580000,0.600000,0.570000,0.640000,0.590000,0.600000,0.610000,0.610000
28,Corp Mgmt (City),Median,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
29,Corp Mgmt (City),Std. Dev.,0.500000,0.490000,0.490000,0.480000,0.480000,0.480000,0.480000,0.490000,0.490000,0.500000,0.480000,0.490000,0.490000,0.490000,0.490000
36,Flood Risk (First Street),Mean,2.730000,2.880000,2.840000,2.320000,2.890000,2.650000,2.900000,4.040000,3.430000,4.830000,4.450000,4.730000,3.670000,3.530000,3.600000
37,Flood Risk (First Street),Median,3.000000,3.000000,3.000000,1.000000,3.000000,1.000000,3.000000,5.000000,4.000000,5.000000,5.000000,5.000000,4.000000,4.000000,4.000000
38,Flood Risk (First Street),Std. Dev.,1.810000,2.000000,1.960000,1.730000,2.050000,1.940000,1.980000,1.980000,2.060000,1.360000,1.780000,1.500000,1.970000,2.080000,2.030000
24,Frac. Corp Owners,Mean,0.210000,0.130000,0.150000,0.150000,0.120000,0.140000,0.150000,0.090000,0.120000,0.230000,0.090000,0.190000,0.200000,0.110000,0.160000


In [2]:
from collections import OrderedDict

# Variables to round as integers
int_vars = {
    'Units',
    'Buildings',
    'Stories',
    'Unit Sqft',
    'Year Built',
    'Flood Risk (First Street)',
    'Per Capita Income (ZIP)',
    'Med Income (Nonprimary)'
}

# All other variables will be rounded to 2 decimals


# Step 1: Extract column info
fixed_cols = ['Variable', 'Statistic']
data_cols = [col for col in summary_df.columns if col not in fixed_cols]

# Parse multilevel headers
col_tuples = [tuple(col.split('|')) for col in data_cols]
col_tuples = [(s.strip(), c.strip()) for s, c in col_tuples]

# Group by story level for multicolumn construction
story_order = ['1-2', '3-4', '5-9', '10+', 'All']
county_order = ['Miami/Broward', 'Other', 'All']
ordered_cols = [(s, c) for s in story_order for c in county_order if (s, c) in col_tuples]

# Reorder columns
new_columns = fixed_cols + [f'{s} | {c}' for s, c in ordered_cols]
summary_df = summary_df[new_columns]

# Step 2: Round numeric values
# for col in summary_df.columns:
#     if col not in fixed_cols:
#         summary_df[col] = pd.to_numeric(summary_df[col], errors='coerce').round(2)
for col in summary_df.columns:
    if col in ['Variable', 'Statistic']:
        continue

    for var in summary_df['Variable'].unique():
        mask = summary_df['Variable'] == var

        if var in int_vars:
            summary_df.loc[mask, col] = summary_df.loc[mask, col].apply(
                lambda x: (
                    f"{int(round(v))}"
                    if pd.notna(v := pd.to_numeric(x, errors="coerce"))
                    else ""
                )
            )
        else:
            summary_df.loc[mask, col] = summary_df.loc[mask, col].apply(
                lambda x: (
                    f"{v:.2f}"
                    if pd.notna(v := pd.to_numeric(x, errors="coerce"))
                    else ""
                )
            )

    
# for col in summary_df.columns:
#     if col in ['Variable', 'Statistic']:
#         continue

#     for var in summary_df['Variable'].unique():
#         mask = summary_df['Variable'] == var

#         if var in int_vars:
#             # Round and convert to int, then to string to avoid ".0"
#             summary_df.loc[mask, col] = (
#                 summary_df.loc[mask, col]
#                 .apply(lambda x: f"{int(round(x))}" if pd.notna(x) else '')
#             )
#         else:
#             # Round to 2 decimals and format as string
#             summary_df.loc[mask, col] = (
#                 summary_df.loc[mask, col]
#                 .apply(lambda x: f"{x:.2f}" if pd.notna(x) else '')
#             )

# Step 3: Generate LaTeX with custom multicolumn header
def generate_latex_table(df):
    lines = []
    lines.append(r'\begin{tabular}{ll' + 'c' * len(ordered_cols) + '}')
    lines.append(r'\toprule')

    # First header row: StoryGroup
    story_counts = OrderedDict()
    for s, _ in ordered_cols:
        story_counts[s] = story_counts.get(s, 0) + 1
    header1 = ['Variable', 'Statistic'] + [
        rf'\multicolumn{{{story_counts[s]}}}{{c}}{{{s} Stories}}' for s in story_counts
    ]
    lines.append(' & '.join(header1) + r' \\')
    lines.append(r'\cmidrule(lr){3-' + f'{2+len(ordered_cols)}' + '}')

    # Second header row: CountyGroup
    header2 = [' ', ' '] + [c for _, c in ordered_cols]
    lines.append(' & '.join(header2) + r' \\')
    lines.append(r'\midrule')

    # Data rows
    for _, row in df.iterrows():
        row_vals = [str(row[col]) for col in df.columns]
        lines.append(' & '.join(row_vals) + r' \\')

    lines.append(r'\bottomrule')
    lines.append(r'\end{tabular}')
    return '\n'.join(lines)

# Export LaTeX table
latex_code = generate_latex_table(summary_df)
with open('summary_stats_twolevel_header.tex', 'w') as f:
    f.write(latex_code)
